# Importing and Querying Data with PostgreSQL (Northwind Database)

Tutorials: https://www.youtube.com/watch?v=zpnHsWOy0RY&list=PLP9IO4UYNF0UQkBXlTMSw0CYsxv-GDkkI

All data and many examples of SQL-Queries are available here: https://www.w3schools.com/sql/default.asp

## Frequently used SQL clauses (for in-depth learning)

<div>
  <style>
    .sql-keyword {
      color: blue;
    }
  </style>
  <span class="sql-keyword">SELECT</span><br>
  <span class="sql-keyword">FROM</span><br>
  <span class="sql-keyword">WHERE</span><br>
  <span class="sql-keyword">GROUP BY</span><br>
  <span class="sql-keyword">HAVING</span><br>
  <span class="sql-keyword">ORDER BY</span><br>
  <span class="sql-keyword">FULL JOIN (FULL OUTER JOIN)</span><br>
  <span class="sql-keyword">INNER JOIN</span><br>
  <span class="sql-keyword">LEFT JOIN</span><br>
  <span class="sql-keyword">RIGHT JOIN</span><br>
  <span class="sql-keyword">ON</span><br>
  <span class="sql-keyword">LIMIT</span><br>
  <span class="sql-keyword">AS</span><br>
  <span class="sql-keyword">AVG</span><br>
  <span class="sql-keyword">COUNT</span><br>
  <span class="sql-keyword">MIN</span><br>
  <span class="sql-keyword">MAX</span><br>
  <span class="sql-keyword">SUM</span><br>
  <span class="sql-keyword">BETWEEN</span><br>
  <span class="sql-keyword">IN</span><br>
  <span class="sql-keyword">IS NULL</span>
</div>

## Libraries and Settings

In [ ]:
# Libraries
import os
import pandas as pd
from sqlalchemy import create_engine, text
import matplotlib.pyplot as plt

# Settings
import warnings
warnings.filterwarnings("ignore")

# Current working directory
print(os.getcwd())

## Read data to separate data frames

In [ ]:
# Define path to data
path = 'workspace/Data/'

# Read data from .csv files
df_customers = pd.read_csv(f'{path}Customers.csv', sep=';')
df_categories = pd.read_csv(f'{path}Categories.csv', sep=';')
df_employees = pd.read_csv(f'{path}Employees.csv', sep=';')
df_order_details = pd.read_csv(f'{path}OrderDetails.csv', sep=';')
df_orders = pd.read_csv(f'{path}Orders.csv', sep=';')
df_products = pd.read_csv(f'{path}Products.csv', sep=';')
df_shippers = pd.read_csv(f'{path}Shippers.csv', sep=';')
df_suppliers = pd.read_csv(f'{path}Suppliers.csv', sep=';')

# Change column names to lowercase
df_customers.columns = df_customers.columns.str.lower()
df_categories.columns = df_categories.columns.str.lower()
df_employees.columns = df_employees.columns.str.lower()
df_order_details.columns = df_order_details.columns.str.lower()
df_orders.columns = df_orders.columns.str.lower()
df_products.columns = df_products.columns.str.lower()
df_shippers.columns = df_shippers.columns.str.lower()
df_suppliers.columns = df_suppliers.columns.str.lower()

# Check dimensions of DataFrames
print("Dimensions of DataFrames:")
print(f"Customers: {df_customers.shape}")
print(f"Categories: {df_categories.shape}")
print(f"Employees: {df_employees.shape}")
print(f"OrderDetails: {df_order_details.shape}")
print(f"Orders: {df_orders.shape}")
print(f"Products: {df_products.shape}")
print(f"Shippers: {df_shippers.shape}")
print(f"Suppliers: {df_suppliers.shape}")


## Create database connection

In [ ]:
# Set up database connection
user = "pgadmin"
password = "geheim"
host = "localhost"
port = "5432"
database = "postgres"

# Create Connection URL
db_connection_url = "postgresql://" + user + ":" + password +\
                    "@" + host + ":" + port + "/" + database

# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Test database connection
with engine.connect() as connection:
    result = connection.execute(text('SELECT current_database()'))
    print(result.fetchone())

# Dispose the engine
engine.dispose()

## Write data frames to database tables

In [18]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Write DataFrames to database
df_customers.to_sql('customers', engine, if_exists='replace')
df_categories.to_sql('categories', engine, if_exists='replace')
df_employees.to_sql('employees', engine, if_exists='replace')
df_order_details.to_sql('order_details', engine, if_exists='replace')
df_orders.to_sql('orders', engine, if_exists='replace')
df_products.to_sql('products', engine, if_exists='replace')
df_shippers.to_sql('shippers', engine, if_exists='replace')
df_suppliers.to_sql('suppliers', engine, if_exists='replace')

# Dispose the engine
engine.dispose()

## List tables in the database

In [ ]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Open a connection
with engine.connect() as connection:

    # Execute the query
    result = connection.execute(text("""SELECT table_name
                                        FROM information_schema.tables
                                        WHERE table_schema = 'public'"""))
    
    # Fetch and print the results
    for row in result:
        print(row[0])

# Dispose the engine
engine.dispose()

## Make SQL query to select data

In [ ]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Write data to table
df_sub = pd.read_sql_query('''SELECT 
                                  p.ProductID, 
                                  p.ProductName, 
                                  c.CategoryName,
                                  p.Price
                              FROM Products AS p
                              INNER JOIN 
                                  Categories AS c ON p.CategoryID = c.CategoryID;''', 
                          con=engine)

# Dispose the engine
engine.dispose()

# Show the data
df_sub

## Plot product prices

In [ ]:
# Plot Histogram
fig = plt.figure( figsize=(7,4))
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
n, bins, patches = plt.hist(x=df_sub['price'], 
                            bins=20, 
                            color='#C53115',
                            alpha=1.00, 
                            rwidth=0.95
                   )
plt.grid(True)
plt.ticklabel_format(style='plain')
plt.grid(axis='y', alpha=0.75)

# Set labels
plt.xlabel('Price (USD)', fontsize=10, labelpad=10)
plt.ylabel('Frequency', fontsize=10, labelpad=10)
plt.title('Histogram of product prices', fontsize=12, pad=10)

# Set fontsize of tick labels
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 10)

plt.show()

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')